In [1]:
import pandas as pd
import os
import pymysql
from sqlalchemy import create_engine
from urllib import parse

In [2]:
#查看所有行列
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)


In [3]:
#路径设置
#银联返回表路径
UnionPayData_Folder=  r"..\1、银联返回表"
#管理端导出路径
SystemData_Folder=  r"..\2、管理端导出表"
#退款表路径
refundData_Folder=  r"..\3、退款输出表"

In [4]:
#合并csv文件函数
def File_merge_csv(Folder):  #Folder文件夹路径
    try:
        File_List = []  #存储文件夹下所有文件的路径
        DataFrame_List = []  #存储dataframe
        for root,dirs,files in os.walk(Folder):
            for file in files:
                file_with_path = os.path.join(root,file)
                File_List.append(file_with_path)
                data_original = pd.read_csv(file_with_path,names=["A"])
                DataFrame_List.append(data_original)
        datas = pd.concat(DataFrame_List,axis=0)
        return datas
    except  Exception as e: 
        print(e)

#合并excel文件函数
def File_merge_excel(Folder):  #Folder文件夹路径
    try:
        File_List = []  #存储文件夹下所有文件的路径
        DataFrame_List = []  #存储dataframe
        for root,dirs,files in os.walk(Folder):
            for file in files:
                file_with_path = os.path.join(root,file)
                File_List.append(file_with_path)
                data = pd.read_excel(file_with_path)
                DataFrame_List.append(data)
        datas = pd.concat(DataFrame_List,axis=0)
        return datas
    except  Exception as e: 
        print(e)

In [9]:
#读取银联表格和管理端表格
UnionPayData = File_merge_csv(UnionPayData_Folder)
SystemData = File_merge_excel(SystemData_Folder)
refundData = File_merge_excel(refundData_Folder)

In [6]:
#数据清洗

#银联表数据清理
#银联表格分列
UnionPayData=UnionPayData.iloc[:,0].str.split("|",expand=True)
#银联表格列重命名
UnionPayData.columns = ["渠道商APPKEY","下单时间","支付单号","渠道商订单号","渠道商订单状态","支付金额","订单金额","充值账号","产品类型","运营商","卡性质","清算金额","行业商户代码","收单机构标识码","发送机构标识码","系统跟踪号","交易传输时间"]
#银联表格删除空值行
UnionPayData.dropna(axis=0,subset=['交易传输时间'],inplace=True)
#重置索引
UnionPayData = UnionPayData.reset_index(drop = True)


#管理端表格数据清洗
SystemData.replace("\t","",regex=True,inplace=True)
SystemData = SystemData.rename(columns=lambda x: x.replace("","").replace('\t',''))
#重置索引
SystemData = SystemData.reset_index(drop = True)

#已退款订单数据清洗
refundData = refundData['渠道商订单号']
#重置索引
refundData = refundData.reset_index(drop = True)


In [7]:
#sqlalchemy方式插入数据库
class SqlControl:
    def __init__(self,table,dataframe):

        self.host = '192.168.1.5'
        self.user = "root"
        self.port = "3306"
        self.db="unionpay"
        self.password = parse.quote_plus("yuhuyi94520@")
        self.tb = table
        self.df = dataframe
        self.port='3306'
        conn = create_engine('mysql+pymysql://'+self.user+':'+self.password+'@'+self.host+':'+self.port+'/'+self.db+'?charset=utf8mb4')
        dataframe.to_sql(self.tb, con=conn, if_exists='append',index=False)

In [ ]:
#同步至数据库
SqlControl("uniondata",UnionPayData)
SqlControl("systemdata",SystemData)
SqlControl("refunddata",refundData)

In [ ]:
# class Sql:
#     def __init__(self):#初始化进行连接和游标的创建
#         self.conn = pymysql.connect(host='localhost',user='root',password='yuhuyi94520@',port=3306,database='unionpay',charset='utf8')
#         self.cursor  = self.conn.cursor()
#         print("连接数据库成功")


#     def SqlAdd(self):#增加

#         SqlStatement='insert into uniondata \
#             (Channel_provider_APPKEY","Order_time","Payment_order_number","Channel_Merchant_Order_Number","Channel_Merchant_Order_Status",\
#                         "Payment_amount","Order_amount","Recharge_account","product_type","Operator","Card_nature","Liquidation_amount","Industry_Merchant_Code",\
#                             "Acquirer_institution_Identification","Sending_institution_identification_code","System_tracking_number","Transaction_transmission_time") \
#                 values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

#         for i in range(len)
        

#     def SqlDelete(self):#删除
#         pass

#     def SqlModify(self):#修改
#         pass

#     def SqlQuery(self):#查询
#         pass

#     def __del__(self):#关闭连接
#         self.cursor.close()
#         self.conn.close()
#         print('关闭数据库连接')


    
